In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
%matplotlib inline
import copy
import datetime
from sklearn.utils import shuffle
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
#from catboost import CatBoostRegressor, CatBoostClassifier
import optuna
from optuna.visualization import plot_optimization_history
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
import json
import ast
import time
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

import os
import glob
import gc

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder


In [3]:
# keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, LambdaCallback
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.layers import Input, Layer, Dense, Concatenate, Reshape, Dropout, merge, Add, BatchNormalization, GaussianNoise
from keras.layers.embeddings import Embedding
from keras import backend as K
from keras.layers import Layer
from keras.callbacks import *
import tensorflow as tf
import math

Using TensorFlow backend.


In [4]:
path = 'C:\\Users\\bokhy\\Desktop\\Python\\Python-Projects\\Kaggle\\google-cloud-ncaa-march-madness-2020-division-1-womens-tournament\\'
SeasonCompactResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WRegularSeasonCompactResults.csv'))
SeasonDetailedResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WRegularSeasonDetailedResults.csv'))
TourneyCompactResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneyCompactResults.csv'))
TourneyDetailedResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneyDetailedResults.csv'))
TourneySeeds = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneySeeds.csv'))
GameCities = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WGameCities.csv'))
Cities = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\Cities.csv'))
Teams = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WTeams.csv'))
Conferences = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WTeamConferences.csv'))
Seasons = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WSeasons.csv'))
TourneySlots = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneySlots.csv'))

test = pd.read_csv(os.path.join(path, 'WSampleSubmissionStage1_2020.csv'))

In [5]:
# format ID
test = test.drop(['Pred'], axis=1)
test['Season'] = test['ID'].apply(lambda x: int(x.split('_')[0]))
test['WTeamID'] = test['ID'].apply(lambda x: int(x.split('_')[1]))
test['LTeamID'] = test['ID'].apply(lambda x: int(x.split('_')[2]))

In [6]:
test.head()

,ID,Season,WTeamID,LTeamID
0,2015_3106_3107,2015,3106,3107
1,2015_3106_3110,2015,3106,3110
2,2015_3106_3113,2015,3106,3113
3,2015_3106_3114,2015,3106,3114
4,2015_3106_3116,2015,3106,3116


In [7]:
TourneyDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [8]:
print(TourneyDetailedResults.shape)

(630, 34)


In [9]:
TourneyCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [10]:
print(TourneyCompactResults.shape)

(1386, 8)


# Tournament CSVs

In [11]:
train = pd.merge(TourneyCompactResults,TourneyDetailedResults, how='left', on=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT'])
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print(train.shape)

(1386, 34)


In [13]:
# merge with Game Cities
GameCities = pd.merge(GameCities, Cities, how = 'left', on = 'CityID')

In [14]:
GameCities.head()

,Season,DayNum,WTeamID,LTeamID,CRType,CityID,City,State
0,2010,11,3103,3237,Regular,4002,Akron,OH
1,2010,11,3104,3399,Regular,4085,Corpus Christi,TX
2,2010,11,3110,3224,Regular,4363,Washington,DC
3,2010,11,3111,3267,Regular,4158,Huntington,WV
4,2010,11,3119,3447,Regular,4367,West Point,NY


In [15]:
columns1 = GameCities.columns.difference(train.columns).tolist() + ["Season", "WTeamID", "LTeamID"]

In [16]:
train = train.merge(GameCities[columns1], how="left", on=["Season", "WTeamID", "LTeamID"])

In [17]:
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print(train.shape)

(1429, 38)


In [19]:
# merge with WSeasons
columns2 = Seasons.columns.difference(train.columns).tolist() + ["Season"]
train = train.merge(Seasons[columns2], how="left", on="Season")
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West


In [20]:
print(train.shape)

(1429, 43)


In [21]:
# merge with WTeams
columns3 = Teams.columns.difference(train.columns).tolist()
train = train.merge(Teams[columns3], how="left", left_on=["WTeamID"], right_on=["TeamID"])
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamID,TeamName
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,3104,Alabama
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,3112,Arizona
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,3163,Connecticut
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,3198,Florida Intl
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,3203,G Washington


In [22]:
train.drop(['TeamID'], axis=1, inplace=True)

In [23]:
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington


In [24]:
train = train.merge(Teams[columns3], how="left", left_on=["LTeamID"], right_on=["TeamID"], suffixes=('_W', '_L'))
train.drop(['TeamID'], axis=1, inplace=True)
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama,UNC Greensboro
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona,Santa Clara
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut,Fairfield
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl,Marquette
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington,Georgia


In [25]:
print(train.shape)

(1429, 45)


In [26]:
# merge with WNCAATourneySeeds
columns4 = TourneySeeds.columns.difference(train.columns).tolist() + ['Season']
train = train.merge(TourneySeeds[columns4].drop_duplicates(subset=["Season","TeamID"]),
                    how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
train.drop(['TeamID'], axis=1, inplace=True)

In [27]:
train = train.merge(TourneySeeds[columns4].drop_duplicates(subset=["Season","TeamID"]),
                    how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], suffixes=('_W', '_L'))
train.drop(['TeamID'], axis=1, inplace=True)
train.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L
0,1998,137,3104,94,3422,46,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama,UNC Greensboro,X02,X15
1,1998,137,3112,75,3365,63,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona,Santa Clara,W03,W14
2,1998,137,3163,93,3193,52,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut,Fairfield,W02,W15
3,1998,137,3198,59,3266,45,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl,Marquette,Y07,Y10
4,1998,137,3203,74,3208,72,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington,Georgia,W10,W07


In [28]:
print(train.shape)

(1429, 47)


In [29]:
# Do same with Test set
# merge with Game Cities
columns5 = GameCities.columns.difference(test.columns).tolist() + ["Season", "WTeamID", "LTeamID"]
test = test.merge(GameCities[columns5].drop_duplicates(subset=["Season", "WTeamID", "LTeamID"]),
                  how="left", on=["Season", "WTeamID", "LTeamID"])
test.head()

,ID,Season,WTeamID,LTeamID,CRType,City,CityID,DayNum,State
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN


In [30]:
# merge with WSeasons
columns6 = Seasons.columns.difference(test.columns).tolist() + ["Season"]
test = test.merge(Seasons[columns6].drop_duplicates(subset=["Season"]),
                  how="left", on= "Season")
test.head()

,ID,Season,WTeamID,LTeamID,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity


In [31]:
# merge with WTeams
columns7 = Teams.columns.difference(test.columns).tolist()
test = test.merge(Teams[columns7].drop_duplicates(subset=["TeamID"]),
                  how="left", left_on=["WTeamID"], right_on=["TeamID"])
test.drop(['TeamID'], axis=1, inplace=True)
test = test.merge(Teams[columns7].drop_duplicates(subset=["TeamID"]),
                  how="left", left_on=["LTeamID"], right_on=["TeamID"], suffixes=('_W', '_L'))
test.drop(['TeamID'], axis=1, inplace=True)
test.head()

,ID,Season,WTeamID,LTeamID,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,SUNY Albany
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,American Univ
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arizona St
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Ark Little Rock
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arkansas


In [32]:
# merge with WNCAATourneySeeds
columns8 = TourneySeeds.columns.difference(test.columns).tolist() + ['Season']
test = test.merge(TourneySeeds[columns8].drop_duplicates(subset=["Season","TeamID"]),
                  how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
test.drop(['TeamID'], axis=1, inplace=True)
test = test.merge(TourneySeeds[columns8].drop_duplicates(subset=["Season","TeamID"]),
                  how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], suffixes=('_W', '_L'))
test.drop(['TeamID'], axis=1, inplace=True)
test.head()

,ID,Season,WTeamID,LTeamID,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,SUNY Albany,Y15,X13
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,American Univ,Y15,Z14
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arizona St,Y15,Y03
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Ark Little Rock,Y15,Y11
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arkansas,Y15,Z10


In [33]:
print(test.shape)

(10080, 18)


In [34]:
# Exclude variables that are in Train-set, but does not in Test-set
columns9 = [c for c in train.columns.values.tolist() if c not in test.columns.values.tolist()]
print(columns9)

['WScore', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']


In [35]:
train = train.drop(columns9, axis=1)
train.head()

,Season,DayNum,WTeamID,LTeamID,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L
0,1998,137,3104,3422,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama,UNC Greensboro,X02,X15
1,1998,137,3112,3365,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona,Santa Clara,W03,W14
2,1998,137,3163,3193,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut,Fairfield,W02,W15
3,1998,137,3198,3266,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl,Marquette,Y07,Y10
4,1998,137,3203,3208,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington,Georgia,W10,W07


In [36]:
# to datetime format
train["DayZero"] = pd.to_datetime(train["DayZero"], infer_datetime_format=True)

In [37]:
# sort by date
train = train.sort_values(by=["DayZero", "Season", "DayNum"]).reset_index(drop=True)

In [38]:
train.head()

,Season,DayNum,WTeamID,LTeamID,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L
0,1998,137,3104,3422,NaN,NaN,NaN,NaN,1997-10-27,East,Midwest,Mideast,West,Alabama,UNC Greensboro,X02,X15
1,1998,137,3112,3365,NaN,NaN,NaN,NaN,1997-10-27,East,Midwest,Mideast,West,Arizona,Santa Clara,W03,W14
2,1998,137,3163,3193,NaN,NaN,NaN,NaN,1997-10-27,East,Midwest,Mideast,West,Connecticut,Fairfield,W02,W15
3,1998,137,3198,3266,NaN,NaN,NaN,NaN,1997-10-27,East,Midwest,Mideast,West,Florida Intl,Marquette,Y07,Y10
4,1998,137,3203,3208,NaN,NaN,NaN,NaN,1997-10-27,East,Midwest,Mideast,West,G Washington,Georgia,W10,W07


In [39]:
test.head()

,ID,Season,WTeamID,LTeamID,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,SUNY Albany,Y15,X13
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,American Univ,Y15,Z14
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arizona St,Y15,Y03
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Ark Little Rock,Y15,Y11
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arkansas,Y15,Z10


# Season CSVs

In [40]:
def merge_regularSeason(df, regularSeason):
    df_new = pd.DataFrame()
    for i, season in enumerate(df["Season"].unique()):
        print(season)
        if season <= 1998:
            continue
            
        # split winners and losers (make sure not to use the future data!)
        team_win_score = regularSeason.loc[regularSeason["Season"] <= season, :].groupby(['WTeamID']).agg({'WScore':['sum', 'count', 'var']}).reset_index()
        team_win_score.columns = [' '.join(col).strip() for col in team_win_score.columns.values]
        team_loss_score = regularSeason.loc[regularSeason["Season"] <= season, :].groupby(['LTeamID']).agg({'LScore':['sum', 'count', 'var']}).reset_index()
        team_loss_score.columns = [' '.join(col).strip() for col in team_loss_score.columns.values]
        
        # merge with train 
        team_win_score["Season"] = season
        team_loss_score["Season"] = season
        df_fold = df.loc[df["Season"] == season, :].reset_index(drop=True)
        df_fold = df_fold.merge(team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
        df_fold = df_fold.merge(team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
        df_fold = df_fold.merge(team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
        df_fold = df_fold.merge(team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
        df_fold.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)
        
        # restore
        df_new = pd.concat([df_new, df_fold], ignore_index=True)
    return df_new

In [41]:
train = merge_regularSeason(train, SeasonCompactResults)
test = merge_regularSeason(test, SeasonCompactResults)

1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2015
2016
2017
2018
2019


In [42]:
def addition(table):
    table['x_score'] = table['WScore sum_x'] + table['LScore sum_y']
    table['y_score'] = table['WScore sum_y'] + table['LScore sum_x']
    table['x_count'] = table['WScore count_x'] + table['LScore count_y']
    table['y_count'] = table['WScore count_y'] + table['WScore count_x']
    table['x_var'] = table['WScore var_x'] + table['LScore var_y']
    table['y_var'] = table['WScore var_y'] + table['WScore var_x']
    return table
train = addition(train)
test = addition(test)

In [43]:
train.head()

,Season,DayNum,WTeamID_x,LTeamID_x,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L,WScore sum_x,WScore count_x,WScore var_x,LScore sum_x,LScore count_x,LScore var_x,LScore sum_y,LScore count_y,LScore var_y,WScore sum_y,WScore count_y,WScore var_y,x_score,y_score,x_count,y_count,x_var,y_var
0,1999,137,3104,3212,NaN,NaN,NaN,NaN,1998-10-26,East,Mideast,Midwest,West,Alabama,Grambling,Y05,Y12,3229,40,131.845513,664,10,149.822222,1335,19,84.315789,3389,39,132.041835,4564,4053,59,79,216.161302,263.887348
1,1999,137,3112,3196,NaN,NaN,NaN,NaN,1998-10-26,East,Mideast,Midwest,West,Arizona,Florida,Y06,Y11,3044,38,90.907539,1391,21,86.490476,1090,16,109.450000,3160,40,166.256410,4134,4551,54,78,200.357539,257.163949
2,1999,137,3155,3197,NaN,NaN,NaN,NaN,1998-10-26,East,Mideast,Midwest,West,Clemson,Florida A&M,X02,X15,3520,46,152.255072,1148,18,76.418301,792,13,122.743590,2331,33,76.363636,4312,3479,59,79,274.998662,228.618709
3,1999,137,3161,3169,NaN,NaN,NaN,NaN,1998-10-26,East,Mideast,Midwest,West,Colorado St,CS Northridge,Z02,Z15,4279,52,142.091629,1226,20,110.115789,465,7,269.619048,2401,33,75.126894,4744,3627,59,85,411.710677,217.218523
4,1999,137,3163,3384,NaN,NaN,NaN,NaN,1998-10-26,East,Mideast,Midwest,West,Connecticut,St Francis PA,X01,X16,5116,57,221.224311,1048,18,117.124183,439,6,32.566667,2812,38,119.135135,5555,3860,63,95,253.790977,340.359446


In [44]:
test.head()

,ID,Season,WTeamID_x,LTeamID_x,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L,WScore sum_x,WScore count_x,WScore var_x,LScore sum_x,LScore count_x,LScore var_x,LScore sum_y,LScore count_y,LScore var_y,WScore sum_y,WScore count_y,WScore var_y,x_score,y_score,x_count,y_count,x_var,y_var
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,SUNY Albany,Y15,X13,15765,236,114.03251,13414,248,86.607483,13799,257,129.346547,14676,216,112.759690,29564,28090,493,452,243.379057,226.792200
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,American Univ,Y15,Z14,15765,236,114.03251,13399,236,100.089776,13799,257,129.346547,19628,291,105.372509,29564,33027,493,527,243.379057,219.405019
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arizona St,Y15,Y03,15765,236,114.03251,10932,191,91.338936,13799,257,129.346547,24404,347,112.868635,29564,35336,493,583,243.379057,226.901144
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Ark Little Rock,Y15,Y11,15765,236,114.03251,11356,208,91.169454,13799,257,129.346547,16853,260,97.723953,29564,28209,493,496,243.379057,211.756463
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arkansas,Y15,Z10,15765,236,114.03251,13268,224,113.667841,13799,257,129.346547,22905,310,136.352907,29564,36173,493,546,243.379057,250.385417


In [45]:
# make winner and loser train
train_win = train.copy()
train_los = train.copy()
train_win = train_win[['WTeamID_x','LTeamID_x','Season','DayNum', 'DayZero', 'Seed_W', 'Seed_L', 'x_score', 'y_score', 'x_count', 'y_count', 'x_var', 'y_var']]
train_los = train_los[['WTeamID_x','LTeamID_x','Season','DayNum', 'DayZero','Seed_L', 'Seed_W', 'y_score', 'x_score', 'x_count', 'y_count', 'x_var', 'y_var']]
train_win.columns = ['TeamName_1','TeamName_2','Season','DayNum', 'DayZero','Seed_1', 'Seed_2', 'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']
train_los.columns = ['TeamName_1','TeamName_2','Season','DayNum', 'DayZero','Seed_1', 'Seed_2', 'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']

In [46]:
# same processing for test
test = test[['ID','WTeamID_x','LTeamID_x','Season','DayNum', 'Seed_W', 'Seed_L', 'x_score', 'y_score', 'x_count', 'y_count', 'x_var', 'y_var']]
test.columns = ['ID','TeamName_1','TeamName_2','Season', 'DayNum', 'Seed_1', 'Seed_2', 'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']

In [47]:
def get_seed(x):
    return int(x[1:3])
train_win['Seed_1'] = train_win['Seed_1'].map(lambda x: get_seed(x))
train_win['Seed_2'] = train_win['Seed_2'].map(lambda x: get_seed(x))

train_los['Seed_1'] = train_los['Seed_1'].map(lambda x: get_seed(x))
train_los['Seed_2']= train_los['Seed_2'].map(lambda x: get_seed(x))

test['Seed_1'] = test['Seed_1'].map(lambda x: get_seed(x))
test['Seed_2']= test['Seed_2'].map(lambda x: get_seed(x))

In [48]:
train_win["result"] = 1
train_los["result"] = 0

In [49]:
train = pd.concat((train_win, train_los)).reset_index(drop=True)
train.head()

,TeamName_1,TeamName_2,Season,DayNum,DayZero,Seed_1,Seed_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,result
0,3104,3212,1999,137,1998-10-26,5,12,4564,4053,59,79,216.161302,263.887348,1
1,3112,3196,1999,137,1998-10-26,6,11,4134,4551,54,78,200.357539,257.163949,1
2,3155,3197,1999,137,1998-10-26,2,15,4312,3479,59,79,274.998662,228.618709,1
3,3161,3169,1999,137,1998-10-26,2,15,4744,3627,59,85,411.710677,217.218523,1
4,3163,3384,1999,137,1998-10-26,1,16,5555,3860,63,95,253.790977,340.359446,1


In [50]:
train = train.sort_values(by=['DayZero', 'DayNum']).reset_index(drop=True)
train.head()

,TeamName_1,TeamName_2,Season,DayNum,DayZero,Seed_1,Seed_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,result
0,3104,3212,1999,137,1998-10-26,5,12,4564,4053,59,79,216.161302,263.887348,1
1,3112,3196,1999,137,1998-10-26,6,11,4134,4551,54,78,200.357539,257.163949,1
2,3155,3197,1999,137,1998-10-26,2,15,4312,3479,59,79,274.998662,228.618709,1
3,3161,3169,1999,137,1998-10-26,2,15,4744,3627,59,85,411.710677,217.218523,1
4,3163,3384,1999,137,1998-10-26,1,16,5555,3860,63,95,253.790977,340.359446,1


In [51]:
test.head()

,ID,TeamName_1,TeamName_2,Season,DayNum,Seed_1,Seed_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2
0,2015_3106_3107,3106,3107,2015,NaN,15,13,29564,28090,493,452,243.379057,226.792200
1,2015_3106_3110,3106,3110,2015,NaN,15,14,29564,33027,493,527,243.379057,219.405019
2,2015_3106_3113,3106,3113,2015,NaN,15,3,29564,35336,493,583,243.379057,226.901144
3,2015_3106_3114,3106,3114,2015,NaN,15,11,29564,28209,493,496,243.379057,211.756463
4,2015_3106_3116,3106,3116,2015,NaN,15,10,29564,36173,493,546,243.379057,250.385417


In [52]:
def addition2(table):
    # Seed
    table['Seed_diff'] = table['Seed_1'] - table['Seed_2']
    table['Seed_div'] = table['Seed_1'] / table['Seed_2']
    
    # Score
    table['Score_diff'] = table['Score_1'] - table['Score_2'] 
    table['Score_div'] = table['Score_1'] / table['Score_2'] 
    
    # Count
    table['Count_diff'] = table['Count_1'] - table['Count_2']
    table['Count_div'] = table['Count_1'] / table['Count_2']   

    # Var
    table['Var_diff'] = table['Var_1'] - table['Var_2']
    table['Var_div'] = table['Var_1'] / table['Var_2']  
    
    # Mean
    table['Mean_score1'] = table['Score_1'] / table['Count_1']
    table['Mean_score2'] = table['Score_2'] / table['Count_2']
    table['Mean_score_diff'] = table['Mean_score1'] - table['Mean_score2']
    table['Mean_score_div'] = table['Mean_score1'] / table['Mean_score2']
    
    # FanoFactor
    table['FanoFactor_1'] = table['Var_1'] / table['Mean_score1']
    table['FanoFactor_2'] = table['Var_2'] / table['Mean_score2']    
    table['FanoFactor_diff'] = table['FanoFactor_1'] - table['FanoFactor_2'] 
    table['FanoFactor_div'] = table['FanoFactor_1'] / table['FanoFactor_2']    
    # Drop
    table = table.drop(['Count_1', 'Count_2', 'Var_1', 'Var_2'], axis=1)
    
    table['win14days'] = np.where((table['DayNum']> 118) & (table['Score_1'] > table['Score_2']), 1, 0)
    table['last14days'] = np.where(table['DayNum']> 118,  1, 0)
    
    table = table.drop(['Score_1', 'Score_2'], axis=1)
    
    return table
train = addition2(train)
test = addition2(test)

In [53]:
test = test.drop(['DayNum'],axis=1)

In [54]:
train.head()

,TeamName_1,TeamName_2,Season,DayNum,DayZero,Seed_1,Seed_2,result,Seed_diff,Seed_div,Score_diff,Score_div,Count_diff,Count_div,Var_diff,Var_div,Mean_score1,Mean_score2,Mean_score_diff,Mean_score_div,FanoFactor_1,FanoFactor_2,FanoFactor_diff,FanoFactor_div,win14days,last14days
0,3104,3212,1999,137,1998-10-26,5,12,1,-7,0.416667,511,1.126079,-20,0.746835,-47.726046,0.819142,77.355932,51.303797,26.052135,1.507801,2.794373,5.143622,-2.349249,0.543269,1,1
1,3112,3196,1999,137,1998-10-26,6,11,1,-5,0.545455,-417,0.908372,-24,0.692308,-56.806410,0.779104,76.555556,58.346154,18.209402,1.312093,2.617152,4.407556,-1.790404,0.593788,0,1
2,3155,3197,1999,137,1998-10-26,2,15,1,-13,0.133333,833,1.239437,-20,0.746835,46.379953,1.202870,73.084746,44.037975,29.046771,1.659585,3.762737,5.191399,-1.428662,0.724802,1,1
3,3161,3169,1999,137,1998-10-26,2,15,1,-13,0.133333,1117,1.307968,-26,0.694118,194.492154,1.895376,80.406780,42.670588,37.736191,1.884361,5.120348,5.090591,0.029757,1.005845,1,1
4,3163,3384,1999,137,1998-10-26,1,16,1,-15,0.062500,1695,1.439119,-32,0.663158,-86.568468,0.745656,88.174603,40.631579,47.543024,2.170100,2.878278,8.376722,-5.498445,0.343604,1,1


In [55]:
test.head()

,ID,TeamName_1,TeamName_2,Season,Seed_1,Seed_2,Seed_diff,Seed_div,Score_diff,Score_div,Count_diff,Count_div,Var_diff,Var_div,Mean_score1,Mean_score2,Mean_score_diff,Mean_score_div,FanoFactor_1,FanoFactor_2,FanoFactor_diff,FanoFactor_div,win14days,last14days
0,2015_3106_3107,3106,3107,2015,15,13,2,1.153846,1474,1.052474,41,1.090708,16.586857,1.073137,59.967546,62.146018,-2.178472,0.964946,4.058513,3.649344,0.409169,1.112121,0,0
1,2015_3106_3110,3106,3110,2015,15,14,1,1.071429,-3463,0.895146,-34,0.935484,23.974038,1.109268,59.967546,62.669829,-2.702284,0.956881,4.058513,3.500967,0.557546,1.159255,0,0
2,2015_3106_3113,3106,3113,2015,15,3,12,5.000000,-5772,0.836654,-90,0.845626,16.477912,1.072622,59.967546,60.610635,-0.643089,0.989390,4.058513,3.743586,0.314927,1.084124,0,0
3,2015_3106_3114,3106,3114,2015,15,11,4,1.363636,1355,1.048034,-3,0.993952,31.622594,1.149335,59.967546,56.872984,3.094562,1.054412,4.058513,3.723323,0.335190,1.090025,0,0
4,2015_3106_3116,3106,3116,2015,15,10,5,1.500000,-6609,0.817295,-53,0.902930,-7.006361,0.972018,59.967546,66.250916,-6.283370,0.905158,4.058513,3.779350,0.279163,1.073865,0,0


In [56]:
len(train)

2732

In [57]:
len(test)

10080

In [58]:
def concat_row(r):
    if r['TeamName_1'] < r['TeamName_2']:
        res = str(r['Season'])+"_"+str(r['TeamName_1'])+"_"+str(r['TeamName_2'])
    else:
        res = str(r['Season'])+"_"+str(r['TeamName_2'])+"_"+str(r['TeamName_1'])
    return res

# Delete leaked from train
def delete_leaked_from_df_train(df_train, df_test):
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    df_train_duplicates = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_idx = df_train_duplicates.index.values
    df_train = df_train.drop(df_train_idx)
    df_train = df_train.drop('Concats', axis=1)
    
    return df_train 

In [59]:
train = delete_leaked_from_df_train(train,test)

In [60]:
len(train)

2058

In [61]:
len(test)

10080

In [62]:
train = train.drop(['TeamName_1', 'TeamName_2'],axis=1)

In [63]:
test = test.drop(['TeamName_1', 'TeamName_2'],axis=1)

In [64]:
target = 'result'
features = train.columns.values.tolist()
dropcols = [target,'DayNum', 'Season','DayZero']

In [65]:
features = [f for f in features if f not in dropcols]
print(features)

['Seed_1', 'Seed_2', 'Seed_diff', 'Seed_div', 'Score_diff', 'Score_div', 'Count_diff', 'Count_div', 'Var_diff', 'Var_div', 'Mean_score1', 'Mean_score2', 'Mean_score_diff', 'Mean_score_div', 'FanoFactor_1', 'FanoFactor_2', 'FanoFactor_diff', 'FanoFactor_div', 'win14days', 'last14days']


In [66]:
train.head()

,Season,DayNum,DayZero,Seed_1,Seed_2,result,Seed_diff,Seed_div,Score_diff,Score_div,Count_diff,Count_div,Var_diff,Var_div,Mean_score1,Mean_score2,Mean_score_diff,Mean_score_div,FanoFactor_1,FanoFactor_2,FanoFactor_diff,FanoFactor_div,win14days,last14days
0,1999,137,1998-10-26,5,12,1,-7,0.416667,511,1.126079,-20,0.746835,-47.726046,0.819142,77.355932,51.303797,26.052135,1.507801,2.794373,5.143622,-2.349249,0.543269,1,1
1,1999,137,1998-10-26,6,11,1,-5,0.545455,-417,0.908372,-24,0.692308,-56.806410,0.779104,76.555556,58.346154,18.209402,1.312093,2.617152,4.407556,-1.790404,0.593788,0,1
2,1999,137,1998-10-26,2,15,1,-13,0.133333,833,1.239437,-20,0.746835,46.379953,1.202870,73.084746,44.037975,29.046771,1.659585,3.762737,5.191399,-1.428662,0.724802,1,1
3,1999,137,1998-10-26,2,15,1,-13,0.133333,1117,1.307968,-26,0.694118,194.492154,1.895376,80.406780,42.670588,37.736191,1.884361,5.120348,5.090591,0.029757,1.005845,1,1
4,1999,137,1998-10-26,1,16,1,-15,0.062500,1695,1.439119,-32,0.663158,-86.568468,0.745656,88.174603,40.631579,47.543024,2.170100,2.878278,8.376722,-5.498445,0.343604,1,1


In [67]:
test.head()

,ID,Season,Seed_1,Seed_2,Seed_diff,Seed_div,Score_diff,Score_div,Count_diff,Count_div,Var_diff,Var_div,Mean_score1,Mean_score2,Mean_score_diff,Mean_score_div,FanoFactor_1,FanoFactor_2,FanoFactor_diff,FanoFactor_div,win14days,last14days
0,2015_3106_3107,2015,15,13,2,1.153846,1474,1.052474,41,1.090708,16.586857,1.073137,59.967546,62.146018,-2.178472,0.964946,4.058513,3.649344,0.409169,1.112121,0,0
1,2015_3106_3110,2015,15,14,1,1.071429,-3463,0.895146,-34,0.935484,23.974038,1.109268,59.967546,62.669829,-2.702284,0.956881,4.058513,3.500967,0.557546,1.159255,0,0
2,2015_3106_3113,2015,15,3,12,5.000000,-5772,0.836654,-90,0.845626,16.477912,1.072622,59.967546,60.610635,-0.643089,0.989390,4.058513,3.743586,0.314927,1.084124,0,0
3,2015_3106_3114,2015,15,11,4,1.363636,1355,1.048034,-3,0.993952,31.622594,1.149335,59.967546,56.872984,3.094562,1.054412,4.058513,3.723323,0.335190,1.090025,0,0
4,2015_3106_3116,2015,15,10,5,1.500000,-6609,0.817295,-53,0.902930,-7.006361,0.972018,59.967546,66.250916,-6.283370,0.905158,4.058513,3.779350,0.279163,1.073865,0,0


# Model

In [68]:
class BaseModel(object):

    def __init__(self, train_df, test_df, target, features, categoricals=[], 
                n_splits=3, cv_method="KFold", group=None, task="regression", 
                parameter_tuning=False, seed=42, scaler=None, verbose=True):
        self.train_df = train_df
        self.test_df = test_df
        self.target = target
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.cv_method = cv_method
        self.group = group
        self.task = task
        self.parameter_tuning = parameter_tuning
        self.seed = seed
        self.scaler = scaler
        self.cv = self.get_cv()
        self.verbose = verbose
        self.params = self.get_params()
        self.y_pred, self.score, self.model, self.oof, self.y_val, self.fi_df = self.fit()

    def train_model(self, train_set, val_set):
        raise NotImplementedError

    def get_params(self):
        raise NotImplementedError

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError

    def convert_x(self, x):
        return x

    def calc_metric(self, y_true, y_pred): # this may need to be changed based on the metric of interest
        if self.task == "multiclass":
            return log_loss(y_true, y_pred)
        elif self.task == "binary":
            return log_loss(y_true, y_pred)
#             return roc_auc_score(y_true, y_pred)
        elif self.task == "regression":
            return np.sqrt(mean_squared_error(y_true, y_pred))

    def get_cv(self):
        if self.cv_method == "KFold":
            cv = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
            return cv.split(self.train_df)
        elif self.cv_method == "StratifiedKFold":
            cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
            return cv.split(self.train_df, self.train_df[self.target])
        elif self.cv_method == "TimeSeriesSplit":
            cv = TimeSeriesSplit(max_train_size=None, n_splits=self.n_splits)
            return cv.split(self.train_df)

    def fit(self):
        # initialize
        oof_pred = np.zeros((self.train_df.shape[0], ))
        y_vals = np.zeros((self.train_df.shape[0], ))
        y_pred = np.zeros((self.test_df.shape[0], ))
        if self.group is not None:
            if self.group in self.features:
                self.features.remove(self.group)
            if self.group in self.categoricals:
                self.categoricals.remove(self.group)
        fi = np.zeros((self.n_splits, len(self.features)))

        # scaling, if necessary
        if self.scaler is not None:
            # fill NaN
            numerical_features = [f for f in self.features if f not in self.categoricals]
            self.train_df[numerical_features] = self.train_df[numerical_features].fillna(self.train_df[numerical_features].median())
            self.test_df[numerical_features] = self.test_df[numerical_features].fillna(self.test_df[numerical_features].median())
            self.train_df[self.categoricals] = self.train_df[self.categoricals].fillna(self.train_df[self.categoricals].mode().iloc[0])
            self.test_df[self.categoricals] = self.test_df[self.categoricals].fillna(self.test_df[self.categoricals].mode().iloc[0])

            # scaling
            if self.scaler == "MinMax":
                scaler = MinMaxScaler()
            elif self.scaler == "Standard":
                scaler = StandardScaler()
            df = pd.concat([self.train_df[numerical_features], self.test_df[numerical_features]], ignore_index=True)
            scaler.fit(df[numerical_features])
            x_test = self.test_df.copy()
            x_test[numerical_features] = scaler.transform(x_test[numerical_features])
            x_test = [np.absolute(x_test[i]) for i in self.categoricals] + [x_test[numerical_features]]
        else:
            x_test = self.test_df[self.features]

        # fitting with out of fold
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            # train test split
            x_train, x_val = self.train_df.loc[train_idx, self.features], self.train_df.loc[val_idx, self.features]
            y_train, y_val = self.train_df.loc[train_idx, self.target], self.train_df.loc[val_idx, self.target]

            # fitting & get feature importance
            if self.scaler is not None:
                x_train[numerical_features] = scaler.transform(x_train[numerical_features])
                x_val[numerical_features] = scaler.transform(x_val[numerical_features])
                x_train = [np.absolute(x_train[i]) for i in self.categoricals] + [x_train[numerical_features]]
                x_val = [np.absolute(x_val[i]) for i in self.categoricals] + [x_val[numerical_features]]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model, importance = self.train_model(train_set, val_set)
            fi[fold, :] = importance
            conv_x_val = self.convert_x(x_val)
            y_vals[val_idx] = y_val
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(x_test)
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits
            print('Partial score of fold {} is: {}'.format(fold, self.calc_metric(y_val, oof_pred[val_idx])))
        
        # feature importance data frame
        fi_df = pd.DataFrame()
        for n in np.arange(self.n_splits):
            tmp = pd.DataFrame()
            tmp["features"] = self.features
            tmp["importance"] = fi[n, :]
            tmp["fold"] = n
            fi_df = pd.concat([fi_df, tmp], ignore_index=True)
        gfi = fi_df[["features", "importance"]].groupby(["features"]).mean().reset_index()
        fi_df = fi_df.merge(gfi, on="features", how="left", suffixes=('', '_mean'))

        # outputs
        loss_score = self.calc_metric(y_vals, oof_pred)
        if self.verbose:
            print('Our oof loss score is: ', loss_score)
        return y_pred, loss_score, model, oof_pred, y_vals, fi_df

    def plot_feature_importance(self, rank_range=[1, 50]):
        # plot
        fig, ax = plt.subplots(1, 1, figsize=(10, 20))
        sorted_df = self.fi_df.sort_values(by = "importance_mean", ascending=False).reset_index().iloc[self.n_splits * (rank_range[0]-1) : self.n_splits * rank_range[1]]
        sns.barplot(data=sorted_df, x ="importance", y ="features", orient='h')
        ax.set_xlabel("feature importance")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        return sorted_df

In [69]:
class LgbModel(BaseModel):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        model = lgb.train(self.params, train_set, num_boost_round = 5000, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        fi = model.feature_importance(importance_type="gain")
        return model, fi

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=self.categoricals)
        return train_set, val_set

    def get_params(self):
        # fast fit parameters
        params = {
          'num_leaves': 127,
          'objective': self.task,
          'min_data_in_leaf': 70,
          'max_depth': -1,
          'learning_rate': 0.005,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "verbosity": -1,
          'random_state': 42,
         }

        # list is here: https://lightgbm.readthedocs.io/en/latest/Parameters.html
        if self.task == "regression":
            params["metric"] = "rmse"
        elif self.task == "binary":
            params["metric"] = "binary_logloss" # auc
        elif self.task == "multiclass":
            params["metric"] = "multi_logloss" # cross_entropy, auc_mu
        
        # Bayesian Optimization by Optuna
        if self.parameter_tuning == True:
            # define objective function
            def objective(trial):
                # train, test split
                train_x, test_x, train_y, test_y = train_test_split(self.train_df[self.features], 
                                                                    self.train_df[self.target],
                                                                    test_size=0.3, random_state=self.seed)
                dtrain = lgb.Dataset(train_x, train_y, categorical_feature=self.categoricals)
                dtest = lgb.Dataset(test_x, test_y, categorical_feature=self.categoricals)

                # parameters to be explored
                hyperparams = {'num_leaves': trial.suggest_int('num_leaves', 24, 1024),
                        'boosting_type': 'gbdt',
                        'objective': params["objective"],
                        'metric': params["metric"],
                        'max_depth': trial.suggest_int('max_depth', 4, 30),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
                        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
                        'early_stopping_rounds': 100
                        }

                # LGB
                model = lgb.train(hyperparams, dtrain, valid_sets=dtest, verbose_eval=500)
                pred = model.predict(test_x)
                if (self.task == "binary") | (self.task == "multiclass"):
                    return log_loss(test_y, pred)
                elif self.task == "regression":
                    return np.sqrt(mean_squared_error(test_y, pred))

            # run optimization
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)

            print('Number of finished trials: {}'.format(len(study.trials)))
            print('Best trial:')
            trial = study.best_trial
            print('  Value: {}'.format(trial.value))
            print('  Params: ')
            for key, value in trial.params.items():
                print('    {}: {}'.format(key, value))

            params = trial.params

            # lower learning rate for better accuracy
            params["learning_rate"] = 0.001

            # plot history
            plot_optimization_history(study)

        return params

In [70]:
test["Season"].unique()

array([2015, 2016, 2017, 2018, 2019], dtype=int64)

In [71]:
# label encoding
categoricals = ["win14days", "last14days"]

In [72]:
models = {}
test["Pred"] = 0.5
pred = np.zeros(test.shape[0])
for season in test["Season"].unique():
    print(f"Predicting {season}...")
    lgbm = LgbModel(train.loc[train["Season"] < season, :], test.loc[test["Season"] == season, :], target, features, categoricals=[], n_splits=10, 
                    cv_method="StratifiedKFold", group=None, task="binary", scaler=None, verbose=True)
    # StratifiedKFold,TimeSeriesSplit
    models[season] = lgbm
    test.loc[test["Season"] == season, "Pred"] = lgbm.y_pred

Predicting 2015...
[100]	training's binary_logloss: 0.533879	valid_1's binary_logloss: 0.534065
[200]	training's binary_logloss: 0.452487	valid_1's binary_logloss: 0.46114
[300]	training's binary_logloss: 0.403154	valid_1's binary_logloss: 0.421739
[400]	training's binary_logloss: 0.36762	valid_1's binary_logloss: 0.397692
[500]	training's binary_logloss: 0.341535	valid_1's binary_logloss: 0.385279
[600]	training's binary_logloss: 0.318978	valid_1's binary_logloss: 0.376851
[700]	training's binary_logloss: 0.30085	valid_1's binary_logloss: 0.369637
[800]	training's binary_logloss: 0.284574	valid_1's binary_logloss: 0.367872
[900]	training's binary_logloss: 0.269425	valid_1's binary_logloss: 0.366573
[1000]	training's binary_logloss: 0.255449	valid_1's binary_logloss: 0.368907
[1100]	training's binary_logloss: 0.242711	valid_1's binary_logloss: 0.369007
[1200]	training's binary_logloss: 0.231964	valid_1's binary_logloss: 0.368506
[1300]	training's binary_logloss: 0.221943	valid_1's bina

In [70]:
# other
for season in test["Season"].unique():
    print(f"Predicting {season}...")
    lgbm = LgbModel(train.loc[train["Season"] < season, :], test.loc[test["Season"] == season, :], target, features, categoricals=[], n_splits=10, 
                    cv_method="StratifiedKFold", group=None, task="binary", scaler=None, verbose=True)
    models[season] = lgbm
    test.loc[test["Season"] == season, "Pred"] = lgbm.y_pred

Predicting 2015...


c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's binary_logloss: 0.522125	valid_1's binary_logloss: 0.537485
[200]	training's binary_logloss: 0.430374	valid_1's binary_logloss: 0.461534
[300]	training's binary_logloss: 0.371556	valid_1's binary_logloss: 0.417546
[400]	training's binary_logloss: 0.325797	valid_1's binary_logloss: 0.39032
[500]	training's binary_logloss: 0.287091	valid_1's binary_logloss: 0.368998
[600]	training's binary_logloss: 0.25769	valid_1's binary_logloss: 0.354284
[700]	training's binary_logloss: 0.228618	valid_1's binary_logloss: 0.336529
[800]	training's binary_logloss: 0.204033	valid_1's binary_logloss: 0.324936
[900]	training's binary_logloss: 0.183104	valid_1's binary_logloss: 0.315081
[1000]	training's binary_logloss: 0.165541	valid_1's binary_logloss: 0.30705
[1100]	training's binary_logloss: 0.150778	valid_1's binary_logloss: 0.302575
[1200]	training's binary_logloss: 0.137685	valid_1's binary_logloss: 0.301567
[1300]	training's binary_logloss: 0.125754	valid_1's binary_logloss: 0.30157

c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's binary_logloss: 0.519681	valid_1's binary_logloss: 0.546828
[200]	training's binary_logloss: 0.425988	valid_1's binary_logloss: 0.481869
[300]	training's binary_logloss: 0.363114	valid_1's binary_logloss: 0.443005
[400]	training's binary_logloss: 0.314901	valid_1's binary_logloss: 0.420556
[500]	training's binary_logloss: 0.279546	valid_1's binary_logloss: 0.403941
[600]	training's binary_logloss: 0.248562	valid_1's binary_logloss: 0.383146
[700]	training's binary_logloss: 0.222969	valid_1's binary_logloss: 0.369535
[800]	training's binary_logloss: 0.20062	valid_1's binary_logloss: 0.359997
[900]	training's binary_logloss: 0.18192	valid_1's binary_logloss: 0.354768
[1000]	training's binary_logloss: 0.164333	valid_1's binary_logloss: 0.348462
[1100]	training's binary_logloss: 0.15005	valid_1's binary_logloss: 0.344954
[1200]	training's binary_logloss: 0.136085	valid_1's binary_logloss: 0.343957
[1300]	training's binary_logloss: 0.123814	valid_1's binary_logloss: 0.34513

...

c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's binary_logloss: 0.520995	valid_1's binary_logloss: 0.550529
[200]	training's binary_logloss: 0.428301	valid_1's binary_logloss: 0.482539
[300]	training's binary_logloss: 0.370495	valid_1's binary_logloss: 0.44349
[400]	training's binary_logloss: 0.32689	valid_1's binary_logloss: 0.419965
[500]	training's binary_logloss: 0.291105	valid_1's binary_logloss: 0.40552
[600]	training's binary_logloss: 0.259566	valid_1's binary_logloss: 0.392561
[700]	training's binary_logloss: 0.232466	valid_1's binary_logloss: 0.383419
[800]	training's binary_logloss: 0.208329	valid_1's binary_logloss: 0.379485
[900]	training's binary_logloss: 0.188449	valid_1's binary_logloss: 0.377105
[1000]	training's binary_logloss: 0.170009	valid_1's binary_logloss: 0.375973
[1100]	training's binary_logloss: 0.153917	valid_1's binary_logloss: 0.377917
[1200]	training's binary_logloss: 0.140276	valid_1's binary_logloss: 0.379374
[1300]	training's binary_logloss: 0.128355	valid_1's binary_logloss: 0.38231

c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's binary_logloss: 0.521819	valid_1's binary_logloss: 0.536742
[200]	training's binary_logloss: 0.427774	valid_1's binary_logloss: 0.458685
[300]	training's binary_logloss: 0.36932	valid_1's binary_logloss: 0.409735
[400]	training's binary_logloss: 0.324173	valid_1's binary_logloss: 0.383873
[500]	training's binary_logloss: 0.286741	valid_1's binary_logloss: 0.370743
[600]	training's binary_logloss: 0.257263	valid_1's binary_logloss: 0.364307
[700]	training's binary_logloss: 0.229812	valid_1's binary_logloss: 0.352819
[800]	training's binary_logloss: 0.204941	valid_1's binary_logloss: 0.342704
[900]	training's binary_logloss: 0.185552	valid_1's binary_logloss: 0.336509
[1000]	training's binary_logloss: 0.169126	valid_1's binary_logloss: 0.332739
[1100]	training's binary_logloss: 0.153915	valid_1's binary_logloss: 0.329805
[1200]	training's binary_logloss: 0.140022	valid_1's binary_logloss: 0.327643
[1300]	training's binary_logloss: 0.127479	valid_1's binary_logloss: 0.328

KeyboardInterrupt: 

In [73]:
test['Pred']

0        0.051043
1        0.091716
2        0.000101
3        0.048542
4        0.049260
           ...   
10075    0.761587
10076    0.983058
10077    0.212244
10078    0.243313
10079    0.974453
Name: Pred, Length: 10080, dtype: float64

In [74]:
len(lgbm.y_pred)

2016

In [ ]:
# Prediction

In [ ]:
#h2o = pd.read_csv(os.path.join(path, 'NCAAW_h2o.csv'))

In [75]:
submission = pd.read_csv(os.path.join(path, 'WSampleSubmissionStage1_2020.csv'))
submission['Pred'] = test['Pred']
submission

,ID,Pred
0,2015_3106_3107,0.051043
1,2015_3106_3110,0.091716
2,2015_3106_3113,0.000101
3,2015_3106_3114,0.048542
4,2015_3106_3116,0.049260
...,...,...
10075,2019_3413_3417,0.761587
10076,2019_3413_3460,0.983058
10077,2019_3416_3417,0.212244
10078,2019_3416_3460,0.243313


In [76]:
submission.to_csv('NCAAW_Ensemble.csv', index=False)

In [34]:
Season = pd.merge(SeasonCompactResults, SeasonDetailedResults, how = 'left',
              on=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT'])

In [35]:
Season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,1998,18,3104,91,3202,41,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998,18,3163,87,3221,76,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998,18,3222,66,3261,59,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998,18,3307,69,3365,62,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998,18,3349,115,3411,35,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# split winners and losers
team_win_score = Season.groupby(['Season', 'WTeamID']).agg({'WScore':['sum', 'count', 'var']}).reset_index()
team_win_score.columns = [' '.join(col).strip() for col in team_win_score.columns.values]
team_loss_score = Season.groupby(['Season', 'LTeamID']).agg({'LScore':['sum', 'count', 'var']}).reset_index()
team_loss_score.columns = [' '.join(col).strip() for col in team_loss_score.columns.values]

train = pd.merge(train, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
train = pd.merge(train, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
train = pd.merge(train, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
train = pd.merge(train, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
train.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)

test = pd.merge(test, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
test = pd.merge(test, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
test = pd.merge(test, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
test = pd.merge(test, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
test.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)

In [36]:
# split winners and losers
team_win_score = Season.groupby(['Season', 'WTeamID']).agg({'WScore':['sum', 'count', 'var']}).reset_index()
team_win_score.head()

Season WTeamID WScore                  
                    sum count         var
0   1998    3102    263     4  108.916667
1   1998    3103    842    11   96.872727
2   1998    3104   1657    21  146.190476
3   1998    3106    421     6   78.566667
4   1998    3108    898    12   42.878788

In [37]:
team_win_score.columns = [' '.join(col).strip() for col in team_win_score.columns.values]

In [38]:
team_win_score.head()

,Season,WTeamID,WScore sum,WScore count,WScore var
0,1998,3102,263,4,108.916667
1,1998,3103,842,11,96.872727
2,1998,3104,1657,21,146.190476
3,1998,3106,421,6,78.566667
4,1998,3108,898,12,42.878788


In [39]:
team_loss_score = Season.groupby(['Season', 'LTeamID']).agg({'LScore':['sum', 'count', 'var']}).reset_index()
team_loss_score.columns = [' '.join(col).strip() for col in team_loss_score.columns.values]

In [40]:
del Season
gc.collect()

121

In [41]:
print(team_win_score.shape)

(7337, 5)


In [42]:
print(team_loss_score.shape)
team_loss_score.head()

(7342, 5)


,Season,LTeamID,LScore sum,LScore count,LScore var
0,1998,3102,1112,20,89.515789
1,1998,3103,1166,18,138.771242
2,1998,3104,640,9,131.111111
3,1998,3106,865,15,95.809524
4,1998,3108,662,11,59.363636


In [43]:
# merge with train 
train = pd.merge(train, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
train = pd.merge(train, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
train = pd.merge(train, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
train = pd.merge(train, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
train.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)
train.head()

,Season,DayNum,WTeamID_x,LTeamID_x,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L,WScore sum_x,WScore count_x,WScore var_x,LScore sum_x,LScore count_x,LScore var_x,LScore sum_y,LScore count_y,LScore var_y,WScore sum_y,WScore count_y,WScore var_y
0,1998,137,3104,3422,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama,UNC Greensboro,X02,X15,1657,21,146.190476,496.0,8.0,85.142857,640.0,9.0,131.111111,1514,19,59.561404
1,1998,137,3112,3365,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona,Santa Clara,W03,W14,1708,21,122.433333,387.0,6.0,87.500000,408.0,6.0,154.800000,1757,22,160.504329
2,1998,137,3163,3193,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut,Fairfield,W02,W15,2557,30,197.081609,462.0,8.0,108.214286,139.0,2.0,0.500000,1488,20,106.147368
3,1998,137,3198,3266,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl,Marquette,Y07,Y10,2270,28,113.920635,415.0,6.0,148.966667,67.0,1.0,NaN,1492,20,46.252632
4,1998,137,3203,3208,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington,Georgia,W10,W07,1283,17,110.014706,598.0,9.0,104.527778,535.0,9.0,51.777778,1398,17,100.441176


In [44]:
# merge with test 
test = pd.merge(test, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
test = pd.merge(test, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
test = pd.merge(test, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
test = pd.merge(test, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
test.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)
test.head()

,ID,Season,WTeamID_x,LTeamID_x,CRType,City,CityID,DayNum,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L,WScore sum_x,WScore count_x,WScore var_x,LScore sum_x,LScore count_x,LScore var_x,LScore sum_y,LScore count_y,LScore var_y,WScore sum_y,WScore count_y,WScore var_y
0,2015_3106_3107,2015,3106,3107,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,SUNY Albany,Y15,X13,885,14,32.950549,460.0,8.0,50.857143,807.0,14.0,122.247253,1761,24,122.244565
1,2015_3106_3110,2015,3106,3110,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,American Univ,Y15,Z14,885,14,32.950549,469.0,8.0,29.696429,807.0,14.0,122.247253,1558,24,96.514493
2,2015_3106_3113,2015,3106,3113,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arizona St,Y15,Y03,885,14,32.950549,283.0,5.0,28.300000,807.0,14.0,122.247253,1891,27,126.498575
3,2015_3106_3114,2015,3106,3114,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Ark Little Rock,Y15,Y11,885,14,32.950549,231.0,4.0,130.916667,807.0,14.0,122.247253,1858,28,78.534392
4,2015_3106_3116,2015,3106,3116,NaN,NaN,NaN,NaN,NaN,11/3/2014,Albany,Spokane,Greensboro,OklahomaCity,Alabama St,Arkansas,Y15,Z10,885,14,32.950549,692.0,13.0,83.692308,807.0,14.0,122.247253,1117,17,89.220588


In [45]:
def addition(table):
    table['x_score'] = table['WScore sum_x'] + table['LScore sum_y']
    table['y_score'] = table['WScore sum_y'] + table['LScore sum_x']
    table['x_count'] = table['WScore count_x'] + table['LScore count_y']
    table['y_count'] = table['WScore count_y'] + table['WScore count_x']
    table['x_var'] = table['WScore var_x'] + table['LScore var_y']
    table['y_var'] = table['WScore var_y'] + table['WScore var_x']
    return table
train = addition(train)
test = addition(test)

In [46]:
train.head()

,Season,DayNum,WTeamID_x,LTeamID_x,CRType,City,CityID,State,DayZero,RegionW,RegionX,RegionY,RegionZ,TeamName_W,TeamName_L,Seed_W,Seed_L,WScore sum_x,WScore count_x,WScore var_x,LScore sum_x,LScore count_x,LScore var_x,LScore sum_y,LScore count_y,LScore var_y,WScore sum_y,WScore count_y,WScore var_y,x_score,y_score,x_count,y_count,x_var,y_var
0,1998,137,3104,3422,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Alabama,UNC Greensboro,X02,X15,1657,21,146.190476,496.0,8.0,85.142857,640.0,9.0,131.111111,1514,19,59.561404,2297.0,2010.0,30.0,40,277.301587,205.751880
1,1998,137,3112,3365,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Arizona,Santa Clara,W03,W14,1708,21,122.433333,387.0,6.0,87.500000,408.0,6.0,154.800000,1757,22,160.504329,2116.0,2144.0,27.0,43,277.233333,282.937662
2,1998,137,3163,3193,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Connecticut,Fairfield,W02,W15,2557,30,197.081609,462.0,8.0,108.214286,139.0,2.0,0.500000,1488,20,106.147368,2696.0,1950.0,32.0,50,197.581609,303.228978
3,1998,137,3198,3266,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,Florida Intl,Marquette,Y07,Y10,2270,28,113.920635,415.0,6.0,148.966667,67.0,1.0,NaN,1492,20,46.252632,2337.0,1907.0,29.0,48,NaN,160.173266
4,1998,137,3203,3208,NaN,NaN,NaN,NaN,10/27/1997,East,Midwest,Mideast,West,G Washington,Georgia,W10,W07,1283,17,110.014706,598.0,9.0,104.527778,535.0,9.0,51.777778,1398,17,100.441176,1818.0,1996.0,26.0,34,161.792484,210.455882


In [47]:
# make winner and loser train
train_win = train.copy()
train_los = train.copy()
train_win = train_win[['Seed_W', 'Seed_L', 'TeamName_W', 'TeamName_L', 
                 'x_score', 'y_score', 'x_count', 'y_count', 'x_var', 'y_var']]
train_los = train_los[['Seed_L', 'Seed_W', 'TeamName_L', 'TeamName_W', 
                 'y_score', 'x_score', 'x_count', 'y_count', 'x_var', 'y_var']]
train_win.columns = ['Seed_1', 'Seed_2', 'TeamName_1', 'TeamName_2',
                  'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']
train_los.columns = ['Seed_1', 'Seed_2', 'TeamName_1', 'TeamName_2',
                  'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']

In [48]:
# same processing for test
test = test[['ID', 'Seed_W', 'Seed_L', 'TeamName_W', 'TeamName_L', 
                 'x_score', 'y_score', 'x_count', 'y_count', 'x_var', 'y_var']]
test.columns = ['ID', 'Seed_1', 'Seed_2', 'TeamName_1', 'TeamName_2',
                  'Score_1', 'Score_2', 'Count_1', 'Count_2', 'Var_1', 'Var_2']

In [49]:
def get_seed(x):
    return int(x[1:3])
train_win['Seed_1'] = train_win['Seed_1'].map(lambda x: get_seed(x))
train_win['Seed_2'] = train_win['Seed_2'].map(lambda x: get_seed(x))

train_los['Seed_1'] = train_los['Seed_1'].map(lambda x: get_seed(x))
train_los['Seed_2']= train_los['Seed_2'].map(lambda x: get_seed(x))

test['Seed_1'] = test['Seed_1'].map(lambda x: get_seed(x))
test['Seed_2']= test['Seed_2'].map(lambda x: get_seed(x))

In [50]:
train_win["result"] = 1
train_los["result"] = 0

In [51]:
train = pd.concat((train_win, train_los)).reset_index(drop=True)
train.head()

,Seed_1,Seed_2,TeamName_1,TeamName_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,result
0,2,15,Alabama,UNC Greensboro,2297.0,2010.0,30.0,40,277.301587,205.751880,1
1,3,14,Arizona,Santa Clara,2116.0,2144.0,27.0,43,277.233333,282.937662,1
2,2,15,Connecticut,Fairfield,2696.0,1950.0,32.0,50,197.581609,303.228978,1
3,7,10,Florida Intl,Marquette,2337.0,1907.0,29.0,48,NaN,160.173266,1
4,10,7,G Washington,Georgia,1818.0,1996.0,26.0,34,161.792484,210.455882,1


## IMportant step ## Add more ##

In [52]:
def addition2(table):
    table['Seed_diff'] = table['Seed_1'] - table['Seed_2']
    table['Score_diff'] = table['Score_1'] - table['Score_2']
    table['Count_diff'] = table['Count_1'] - table['Count_2']
    table['Var_diff'] = table['Var_1'] - table['Var_2']
    table['Mean_score1'] = table['Score_1'] / table['Count_1']
    table['Mean_score2'] = table['Score_2'] / table['Count_2']
    table['Mean_score_diff'] = table['Mean_score1'] - table['Mean_score2']
    table['FanoFactor_1'] = table['Var_1'] / table['Mean_score1']
    table['FanoFactor_2'] = table['Var_2'] / table['Mean_score2']    
    return table
train = addition2(train)
test = addition2(test)

################

In [53]:
train.head()

,Seed_1,Seed_2,TeamName_1,TeamName_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,result,Seed_diff,Score_diff,Count_diff,Var_diff,Mean_score1,Mean_score2,Mean_score_diff,FanoFactor_1,FanoFactor_2
0,2,15,Alabama,UNC Greensboro,2297.0,2010.0,30.0,40,277.301587,205.751880,1,-13,287.0,-10.0,71.549708,76.566667,50.250000,26.316667,3.621701,4.094565
1,3,14,Arizona,Santa Clara,2116.0,2144.0,27.0,43,277.233333,282.937662,1,-11,-28.0,-16.0,-5.704329,78.370370,49.860465,28.509905,3.537476,5.674589
2,2,15,Connecticut,Fairfield,2696.0,1950.0,32.0,50,197.581609,303.228978,1,-13,746.0,-18.0,-105.647368,84.250000,39.000000,45.250000,2.345182,7.775102
3,7,10,Florida Intl,Marquette,2337.0,1907.0,29.0,48,NaN,160.173266,1,-3,430.0,-19.0,NaN,80.586207,39.729167,40.857040,NaN,4.031629
4,10,7,G Washington,Georgia,1818.0,1996.0,26.0,34,161.792484,210.455882,1,3,-178.0,-8.0,-48.663399,69.923077,58.705882,11.217195,2.313864,3.584920


In [54]:
# label encoding
categoricals = ["TeamName_1", "TeamName_2"]
for c in categoricals:
    le = LabelEncoder()
    train[c] = train[c].fillna("NaN")
    train[c] = le.fit_transform(train[c])
    test[c] = le.transform(test[c])
train.head()

,Seed_1,Seed_2,TeamName_1,TeamName_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,result,Seed_diff,Score_diff,Count_diff,Var_diff,Mean_score1,Mean_score2,Mean_score_diff,FanoFactor_1,FanoFactor_2
0,2,15,2,226,2297.0,2010.0,30.0,40,277.301587,205.751880,1,-13,287.0,-10.0,71.549708,76.566667,50.250000,26.316667,3.621701,4.094565
1,3,14,7,183,2116.0,2144.0,27.0,43,277.233333,282.937662,1,-11,-28.0,-16.0,-5.704329,78.370370,49.860465,28.509905,3.537476,5.674589
2,2,15,40,62,2696.0,1950.0,32.0,50,197.581609,303.228978,1,-13,746.0,-18.0,-105.647368,84.250000,39.000000,45.250000,2.345182,7.775102
3,7,10,65,115,2337.0,1907.0,29.0,48,NaN,160.173266,1,-3,430.0,-19.0,NaN,80.586207,39.729167,40.857040,NaN,4.031629
4,10,7,70,73,1818.0,1996.0,26.0,34,161.792484,210.455882,1,3,-178.0,-8.0,-48.663399,69.923077,58.705882,11.217195,2.313864,3.584920


In [56]:
test.head()

,ID,Seed_1,Seed_2,TeamName_1,TeamName_2,Score_1,Score_2,Count_1,Count_2,Var_1,Var_2,Seed_diff,Score_diff,Count_diff,Var_diff,Mean_score1,Mean_score2,Mean_score_diff,FanoFactor_1,FanoFactor_2
0,2015_3106_3107,15,13,3,177,1692.0,2221.0,28.0,38,155.197802,155.195115,2,-529.0,-10.0,0.002688,60.428571,58.447368,1.981203,2.568285,2.655297
1,2015_3106_3110,15,14,3,5,1692.0,2027.0,28.0,38,155.197802,129.465042,1,-335.0,-10.0,25.732760,60.428571,53.342105,7.086466,2.568285,2.427070
2,2015_3106_3113,15,3,3,8,1692.0,2174.0,28.0,41,155.197802,159.449125,12,-482.0,-13.0,-4.251323,60.428571,53.024390,7.404181,2.568285,3.007090
3,2015_3106_3114,15,11,3,9,1692.0,2089.0,28.0,42,155.197802,111.484941,4,-397.0,-14.0,43.712861,60.428571,49.738095,10.690476,2.568285,2.241440
4,2015_3106_3116,15,10,3,10,1692.0,1809.0,28.0,31,155.197802,122.171138,5,-117.0,-3.0,33.026665,60.428571,58.354839,2.073733,2.568285,2.093591


In [ ]:
# New Submit

In [20]:
path = 'C:\\Users\\bokhy\\Desktop\\'
train = pd.read_csv(os.path.join(path, 'NCAAW_train.csv'))
test = pd.read_csv(os.path.join(path, 'NCAAW_test.csv'))

In [22]:
class BaseModel(object):
    """
    Base Model Class

    """

    def __init__(self, train_df, test_df, target, features, categoricals=[], 
                n_splits=3, cv_method="KFold", group=None, task="regression", 
                parameter_tuning=False, scaler=None, verbose=True):
        self.train_df = train_df
        self.test_df = test_df
        self.target = target
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.cv_method = cv_method
        self.group = group
        self.task = task
        self.parameter_tuning = parameter_tuning
        self.scaler = scaler
        self.cv = self.get_cv()
        self.verbose = verbose
        self.params = self.get_params()
        self.y_pred, self.score, self.model, self.oof, self.y_val, self.fi_df = self.fit()

    def train_model(self, train_set, val_set):
        raise NotImplementedError

    def get_params(self):
        raise NotImplementedError

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError

    def convert_x(self, x):
        return x

    def calc_metric(self, y_true, y_pred): # this may need to be changed based on the metric of interest
        if self.task == "classification":
            return log_loss(y_true, y_pred)
        elif self.task == "binary":
            return log_loss(y_true, y_pred)
#             return roc_auc_score(y_true, y_pred)
        elif self.task == "regression":
            return np.sqrt(mean_squared_error(y_true, y_pred))

    def get_cv(self):
        if self.cv_method == "KFold":
            cv = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
            return cv.split(self.train_df)
        elif self.cv_method == "StratifiedKFold":
            cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
            return cv.split(self.train_df, self.train_df[self.target])
        elif self.cv_method == "TimeSeriesSplit":
            cv = TimeSeriesSplit(max_train_size=None, n_splits=self.n_splits)
            return cv.split(self.train_df)
        elif self.cv_method == "GroupKFold":
            cv = GroupKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
            return cv.split(self.train_df, self.train_df[self.target], self.group)
        elif self.cv_method == "StratifiedGroupKFold":
            cv = StratifiedGroupKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
            return cv.split(self.train_df, self.train_df[self.target], self.group)

    def fit(self):
        # initialize
        oof_pred = np.zeros((self.train_df.shape[0], ))
        y_vals = np.zeros((self.train_df.shape[0], ))
        y_pred = np.zeros((self.test_df.shape[0], ))
        if self.group is not None:
            if self.group in self.features:
                self.features.remove(self.group)
            if self.group in self.categoricals:
                self.categoricals.remove(self.group)
        fi = np.zeros((self.n_splits, len(self.features)))

        # scaling, if necessary
        if self.scaler is not None:
            # fill NaN
            numerical_features = [f for f in self.features if f not in self.categoricals]
            self.train_df[numerical_features] = self.train_df[numerical_features].fillna(self.train_df[numerical_features].median())
            self.test_df[numerical_features] = self.test_df[numerical_features].fillna(self.test_df[numerical_features].median())
            self.train_df[self.categoricals] = self.train_df[self.categoricals].fillna(self.train_df[self.categoricals].mode().iloc[0])
            self.test_df[self.categoricals] = self.test_df[self.categoricals].fillna(self.test_df[self.categoricals].mode().iloc[0])

            # scaling
            if self.scaler == "MinMax":
                scaler = MinMaxScaler()
            elif self.scaler == "Standard":
                scaler = StandardScaler()
            df = pd.concat([self.train_df[numerical_features], self.test_df[numerical_features]], ignore_index=True)
            scaler.fit(df[numerical_features])
            x_test = self.test_df.copy()
            x_test[numerical_features] = scaler.transform(x_test[numerical_features])
            x_test = [np.absolute(x_test[i]) for i in self.categoricals] + [x_test[numerical_features]]
        else:
            x_test = self.test_df[self.features]
            
        # fitting with out of fold
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            # train test split
            x_train, x_val = self.train_df.loc[train_idx, self.features], self.train_df.loc[val_idx, self.features]
            y_train, y_val = self.train_df.loc[train_idx, self.target], self.train_df.loc[val_idx, self.target]

            # fitting & get feature importance
            if self.scaler is not None:
                x_train[numerical_features] = scaler.transform(x_train[numerical_features])
                x_val[numerical_features] = scaler.transform(x_val[numerical_features])
                x_train = [np.absolute(x_train[i]) for i in self.categoricals] + [x_train[numerical_features]]
                x_val = [np.absolute(x_val[i]) for i in self.categoricals] + [x_val[numerical_features]]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model, importance = self.train_model(train_set, val_set)
            fi[fold, :] = importance
            conv_x_val = self.convert_x(x_val)
            y_vals[val_idx] = y_val
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(x_test)
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits
            print('Partial score of fold {} is: {}'.format(fold, self.calc_metric(y_val, oof_pred[val_idx])))

        # feature importance data frame
        fi_df = pd.DataFrame()
        for n in np.arange(self.n_splits):
            tmp = pd.DataFrame()
            tmp["features"] = self.features
            tmp["importance"] = fi[n, :]
            tmp["fold"] = n
            fi_df = pd.concat([fi_df, tmp], ignore_index=True)
        gfi = fi_df[["features", "importance"]].groupby(["features"]).mean().reset_index()
        fi_df = fi_df.merge(gfi, on="features", how="left", suffixes=('', '_mean'))

        # outputs
        loss_score = self.calc_metric(self.train_df[self.target], oof_pred)
        if self.verbose:
            print('Our oof loss score is: ', loss_score)
        return y_pred, loss_score, model, oof_pred, y_vals, fi_df

    def plot_feature_importance(self, rank_range=[1, 50]):
        # plot
        fig, ax = plt.subplots(1, 1, figsize=(10, 20))
        sorted_df = self.fi_df.sort_values(by = "importance_mean", ascending=False).reset_index().iloc[self.n_splits * (rank_range[0]-1) : self.n_splits * rank_range[1]]
        sns.barplot(data=sorted_df, x ="importance", y ="features", orient='h')
        ax.set_xlabel("feature importance")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        return sorted_df

In [23]:
class LgbModel(BaseModel):

    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        model = lgb.train(self.params, train_set, num_boost_round = 10000, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        fi = model.feature_importance(importance_type="gain")
        return model, fi

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=self.categoricals)
        return train_set, val_set

    def get_params(self):
        # params from https://www.kaggle.com/vbmokin/mm-2020-ncaam-simple-lightgbm-on-kfold-tuning
        params = {
          'objective': self.task,
          'num_leaves': 127,
          'min_data_in_leaf': 50,
          'max_depth': -1,
          'learning_rate': 0.005,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "verbosity": -1,
          'random_state': 42,
         }
        
        if self.task == "regression":
            params["objective"] = "regression"
            params["metric"] = "rmse"
        elif self.task == "classification":
            params["objective"] = "binary"
            params["metric"] = "binary_logloss"
        
        # Bayesian Optimization by Optuna
        if self.parameter_tuning == True:
            # define objective function
            def objective(trial):
                # train, test split
                train_x, test_x, train_y, test_y = train_test_split(self.train_df[self.features], 
                                                                    self.train_df[self.target],
                                                                    test_size=0.3, random_state=42)
                dtrain = lgb.Dataset(train_x, train_y, categorical_feature=self.categoricals)
                dtest = lgb.Dataset(test_x, test_y, categorical_feature=self.categoricals)

                # parameters to be explored
                hyperparams = {'num_leaves': trial.suggest_int('num_leaves', 24, 1024),
                        'boosting_type': 'gbdt',
                        'objective': params["objective"],
                        'metric': params["metric"],
                        'max_depth': trial.suggest_int('max_depth', 1, 30),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
                        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
                        'early_stopping_rounds': 100
                        }

                # LGB
                model = lgb.train(hyperparams, dtrain, valid_sets=dtest, verbose_eval=500)
                pred = model.predict(test_x)
                if self.task == "classification":
                    return log_loss(test_y, pred)
                elif self.task == "regression":
                    return np.sqrt(mean_squared_error(test_y, pred))

            # run optimization
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)

            print('Number of finished trials: {}'.format(len(study.trials)))
            print('Best trial:')
            trial = study.best_trial
            print('  Value: {}'.format(trial.value))
            print('  Params: ')
            for key, value in trial.params.items():
                print('    {}: {}'.format(key, value))

            params = trial.params

            # lower learning rate for better accuracy
            params["learning_rate"] = 0.001

            # plot history
            plot_optimization_history(study)

        return params


In [ ]:
# XGB

In [41]:
X = train.drop('result', axis=1)
y = train.result
testdf = test.drop('ID', axis = 1)

In [42]:
features = X.columns.values.tolist()

In [44]:
def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(X[features], y, test_size=0.2)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    param = {
        'silent': 1,
        'objective': 'binary:logistic',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
    }

    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 100)
        param['n_estimators'] = trial.suggest_int('n_estimators', 0, 10000)
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 20)
        param['scale_pos_weight'] = trial.suggest_int('scale_pos_weight', 1, 100)
#        param['subsample'] = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
#        param['colsample_bytree'] = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)
        param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds)
    mse = sklearn.metrics.mean_squared_error(test_y, pred_labels)
    return mse

In [45]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)
print(study.best_trial)

c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
[I 2020-02-22 12:05:06,602] Finished trial#0 resulted in value: 0.1958041958041958. Current best value is 0.1958041958041958 with parameters: {'booster': 'gblinear', 'lambda': 5.0917321013677575e-05, 'alpha': 3.5617545288722005e-08}.
c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
[I 2020-02-22 12:05:06,692] Finished trial#1 resulted in value: 0.23251748251748253. Current best value is 0.1958041958041958 with parameters: {'booster': 'gblinear', 'lambda': 5.0917321013677575e-05, 'alpha': 3.5617545288722005e-08}.
c:\users\bokhy\appdata\local\programs\python\python37\lib\site-packages\xgboost\c

FrozenTrial(number=96, value=0.17307692307692307, datetime_start=datetime.datetime(2020, 2, 22, 12, 5, 17, 106153), datetime_complete=datetime.datetime(2020, 2, 22, 12, 5, 17, 200900), params={'booster': 'gblinear', 'lambda': 0.00020707899904106472, 'alpha': 1.7315145105803258e-05}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08)}, user_attrs={}, system_attrs={'_number': 96}, intermediate_values={}, trial_id=96, state=TrialState.COMPLETE)


In [46]:
trial = study.best_trial
params = trial.params
params

{'booster': 'gblinear',
 'lambda': 0.00020707899904106472,
 'alpha': 1.7315145105803258e-05}

In [59]:
params_xgb = {'colsample_bytree': 0.8,                 
              'learning_rate': 0.001,
              'max_depth': 30,
              'subsample': 1,
              'objective':'binary:logistic',
              'eval_metric':'logloss',
              'min_child_weight':2,
              'gamma':0.25,
              'n_estimators':5000
              }

In [60]:
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [62]:
NFOLDS = 20
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_xgb_preds = np.zeros(testdf.shape[0])
y_oof = np.zeros(X.shape[0])

  
for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_set = xgb.DMatrix(X_train, y_train)
    val_set = xgb.DMatrix(X_valid, y_valid)
    test_set = xgb.DMatrix(testdf)

    clf = xgb.train(params_xgb, train_set, num_boost_round=10000, evals=[(train_set, 'train'), (val_set, 'val')], early_stopping_rounds=100, verbose_eval=200)
    
    y_xgb_preds += clf.predict(test_set) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

Fold: 1
[0]	train-logloss:0.693086	val-logloss:0.693109
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.681367	val-logloss:0.686901
[400]	train-logloss:0.669977	val-logloss:0.680923
[600]	train-logloss:0.658893	val-logloss:0.675317
[800]	train-logloss:0.648149	val-logloss:0.669869
[1000]	train-logloss:0.637729	val-logloss:0.664652
[1200]	train-logloss:0.627616	val-logloss:0.659624
[1400]	train-logloss:0.617774	val-logloss:0.654804
[1600]	train-logloss:0.608171	val-logloss:0.650146
[1800]	train-logloss:0.598819	val-logloss:0.645573
[2000]	train-logloss:0.589721	val-logloss:0.6412
[2200]	train-logloss:0.580874	val-logloss:0.636962
[2400]	train-logloss:0.572256	val-logloss:0.632913
[2600]	train-logloss:0.563847	val-logloss:0.629109
[2800]	train-logloss:0.555667	val-logloss:0.625406
[3000]	train-logloss:0.547704	val-logloss:0.621777
[3200]	train-logloss:0.539927	val-log

In [63]:
print(len(y_xgb_preds))

10080


In [68]:
y_xgb_preds

array([0.4662427 , 0.47839195, 0.19164555, ..., 0.26011047, 0.52683154,
       0.68783275])

In [ ]:
# h2o

In [31]:
h2o = pd.read_csv(os.path.join(path, 'NCAAW_h2o.csv'))
h2o.head()

,ID,Pred
0,2015_3106_3107,0.337843
1,2015_3106_3110,0.392903
2,2015_3106_3113,0.040393
3,2015_3106_3114,0.302683
4,2015_3106_3116,0.179953


In [64]:
path = 'C:\\Users\\bokhy\\Desktop\\kaggle\\google-cloud-ncaa-march-madness-2020-division-1-womens-tournament\\'

In [69]:
submission = pd.read_csv(os.path.join(path, 'WSampleSubmissionStage1_2020.csv'))
submission['Pred'] = 0.85*lgbm.y_pred + 0.15*h2o['Pred'] 
#submission['Pred'] = lgbm.y_pred
submission

,ID,Pred
0,2015_3106_3107,0.230428
1,2015_3106_3110,0.374999
2,2015_3106_3113,0.023223
3,2015_3106_3114,0.198371
4,2015_3106_3116,0.058016
...,...,...
10075,2019_3413_3417,0.030469
10076,2019_3413_3460,0.598030
10077,2019_3416_3417,0.037444
10078,2019_3416_3460,0.798851


In [70]:
submission.to_csv('NCAAW_Ensemble.csv', index=False)